In [103]:
import pandas as pd
import numpy as np

### Preprocessing for all possible TABLES

In [104]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [105]:
import pandas as pd
from pandas_profiling import ProfileReport
profile = ProfileReport(df)

In [106]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [107]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [108]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_7708\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [109]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [110]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [111]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [112]:
df.loc[df["MODO"]=="1", "MODO"]  = "0"
df.loc[df["MODO"]=="2", "MODO"]  = "1"

In [113]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,76953,515,4,660,26,1,0,1,0.0000,22215.0,660.000000,0.00000
1,14,84318,515,4,660,26,1,0,1,0.0000,22215.0,660.000000,0.00000
2,14,85579,340,4,520,32,1,0,1,0.0000,20978.0,520.000000,0.00000
3,14,69671,340,4,520,32,1,0,1,0.0000,40722.0,520.000000,0.00000
4,14,14001,852,14,650-660,81,0,0,1,0.0000,14126.0,654.501718,9.19244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,1,0,1,3.6800,NaN,700.000000,0.00000
9597,22,58769,239,6,700,32,1,0,1,4.2500,NaN,700.000000,0.00000
9598,22,58769,239,6,700,59,1,0,1,4.0700,NaN,700.000000,0.00000
9599,22,88928,239,6,700,40,1,0,1,4.5727,NaN,700.000000,0.00000


Al fer servir producció com a feature, no podrem calcular la superfície de CAMPAÑA. S'haurà de fer manualment.

In [115]:
df2 = df.drop(index=df[df["CAMPAÑA"] == "22"].index).copy()

### 1 - Table without SUPERFICIE

In [ ]:
df_temporal = df2.drop(axis = 1, columns = ["SUPERFICIE"])

In [ ]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)
encoder = ce.OrdinalEncoder(cols = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)

In [ ]:
df_temporal.to_csv("D_T_1.csv")
df_encoded.to_csv("D_T_1_encoded.csv")

### 2 - Table without CAMPAÑA and SUPERFICIE

In [ ]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "SUPERFICIE"])

In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)

In [ ]:
df_temporal.to_csv("D_T_2.csv")
df_encoded.to_csv("D_T_encoded_2.csv")

### 3 - Table without CAMPAÑA. SUPERFICIE PREDICTION

#### X, y, X_train, y_train, X_test, y_test

In [116]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD", "COLOR", "TIPO", "MODO"])

In [117]:
df_temporal = df2.drop(axis = 1, columns = ["CAMPAÑA"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

C:\Users\xavid\AppData\Local\Temp\ipykernel_7708\586565245.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
C:\Users\xavid\AppData\Local\Temp\ipykernel_7708\586565245.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]


In [118]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)

In [119]:
"""from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

normalizer = StandardScaler()
norm_X = normalizer.fit_transform(X)
norm_to_predict = normalizer.transform(to_predict)"""

'from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler\n\nnormalizer = StandardScaler()\nnorm_X = normalizer.fit_transform(X)\nnorm_to_predict = normalizer.transform(to_predict)'

#### xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
gsc = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300, 500, 1000],
                        "learning_rate": [0.15],
                        "max_depth": [40, 60, 100],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result = gsc.fit(X, y)

In [ ]:
print(grid_result.best_params_)
df_grid = pd.DataFrame(grid_result.cv_results_)
df_grid[df_grid["mean_test_score"]>0.86]

#### Evaluating r2score

In [ ]:
import xgboost as xgb
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBRegressor(**params_model)
model.fit(X_train,y_train)
model.score(X_test, y_test)

#### Predicting rows with no SUPERFICIE

In [84]:
#params_model = grid_result.best_params_
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBRegressor(**params_model)
model.fit(X,y)
predicted = model.predict(to_predict)

In [85]:
final_predict = df2[~((df["PRODUCCION"] != 0) & (df2["SUPERFICIE"] != 0))]
final_predict["SUPERFICIE"] = predicted
final_train = df2[(df2["PRODUCCION"] != 0) & (df2["SUPERFICIE"] != 0)]

C:\Users\xavid\AppData\Local\Temp\ipykernel_7708\193630758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_predict["SUPERFICIE"] = predicted


In [121]:
final_df = pd.concat([final_predict, final_train])

In [ ]:
final_df.to_csv("./ds_tractats/df_train_tractat.csv", index = False)

In [140]:
final_df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,76953,515,4,660,26,1,0,1,8.123419,22215.0,660.000000,0.00000
1,14,84318,515,4,660,26,1,0,1,8.123419,22215.0,660.000000,0.00000
2,14,85579,340,4,520,32,1,0,1,6.600629,20978.0,520.000000,0.00000
3,14,69671,340,4,520,32,1,0,1,7.479296,40722.0,520.000000,0.00000
4,14,14001,852,14,650-660,81,0,0,1,6.609812,14126.0,654.501718,9.19244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,21,37461,239,6,700,52,1,0,1,3.680000,28160.1,700.000000,0.00000
8522,21,58769,239,6,700,32,1,0,1,4.250000,41310.0,700.000000,0.00000
8523,21,58769,239,6,700,59,1,0,1,4.160000,45420.0,700.000000,0.00000
8524,21,88928,239,6,700,40,1,0,1,4.750000,56140.0,700.000000,0.00000


### Obtain CAMPAÑA 2022 WITHOUT NANS IN SUPERFICIE

In [172]:
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD", "COLOR", "TIPO", "MODO"])
encoder.fit(df)

OrdinalEncoder(cols=['ID_FINCA', 'ID_ZONA', 'ID_ESTACION', 'VARIEDAD',
                     'ALTITUD', 'COLOR', 'TIPO', 'MODO'],
               mapping=[{'col': 'ID_FINCA', 'data_type': dtype('O'),
                         'mapping': 76953       1
84318       2
85579       3
69671       4
14001       5
         ... 
61497    1228
702      1229
96832    1230
83256    1231
NaN        -2
Length: 1232, dtype: int64},
                        {'col': 'ID_ZONA', 'data_type': dtype('O'),
                         'mapping': 515      1
340      2
852      3
602      4
215      5
      ... 
62     122
816    123
772    124
233...
NaN    -2
dtype: int64},
                        {'col': 'ALTITUD', 'data_type': dtype('O'),
                         'mapping': 660         1
520         2
650-660     3
690         4
695         5
           ..
390        82
385        83
525        84
440        85
NaN        -2
Length: 86, dtype: int64},
                        {'col': 'COLOR', 'data_type': dtype('O'),
                         'mapping': 1      1
0      2
NaN   -2
dtype: int64},
                        {'col': 'TIPO', 'data_type': dtype('O'),
                         'mapping': 0      1
1      2
NaN   -2
dtype: int64},
                        {'col': 'MODO', 'data_type': dtype('O'),
                         'mapping': 1      1
0      2
NaN   -2
dtype: int64}])

In [173]:
df_c_train = df[(df_encoded["CAMPAÑA"] == "22") & (df["SUPERFICIE"] != 0)]
df_c_predict = df[(df_encoded["CAMPAÑA"] == "22") & (df["SUPERFICIE"] == 0)]
df_c_train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
8526,22,48626,302,13,600,32,1,0,1,3.7503,NaN,600.0,0.0
8527,22,47921,302,13,600,32,1,0,1,7.3735,NaN,600.0,0.0
8528,22,5696,919,14,650-660,59,0,0,1,7.6200,NaN,650.0,10.0
8529,22,98814,919,14,650-660,32,1,0,1,3.3267,NaN,650.0,10.0
8530,22,98814,919,14,650-660,40,1,0,1,2.7724,NaN,650.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,1,0,1,3.6800,NaN,700.0,0.0
9597,22,58769,239,6,700,32,1,0,1,4.2500,NaN,700.0,0.0
9598,22,58769,239,6,700,59,1,0,1,4.0700,NaN,700.0,0.0
9599,22,88928,239,6,700,40,1,0,1,4.5727,NaN,700.0,0.0


In [175]:
to_train_22 = pd.concat([encoder.transform(final_df), encoder.transform(df_c_train)])
to_predict_22 = encoder.transform(df_c_predict)

In [180]:
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBRegressor(**params_model)
model.fit(to_train_22.drop(columns=["CAMPAÑA", "PRODUCCION"]),to_train_22["SUPERFICIE"])
predicted = model.predict(to_predict_22.drop(columns=["CAMPAÑA", "PRODUCCION"]))

In [181]:
df_c_predict["SUPERFICIE"] = predicted
df_c_predict

C:\Users\xavid\AppData\Local\Temp\ipykernel_7708\4045202674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c_predict["SUPERFICIE"] = predicted


,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
8536,22,4024,919,14,650-660,59,1,0,1,3.729101,NaN,650.0,10.0
8544,22,19067,919,14,650-660,68,0,0,0,0.686300,NaN,650.0,10.0
8592,22,86582,464,14,650-660,17,0,0,1,5.161902,NaN,650.0,10.0
8593,22,86582,464,14,650-660,59,0,0,1,5.949507,NaN,650.0,10.0
8598,22,77984,464,14,650-660,59,1,0,1,0.873898,NaN,650.0,10.0
8601,22,13054,464,14,650-660,59,1,0,1,1.096758,NaN,650.0,10.0
8606,22,16692,464,14,650-660,59,1,0,1,1.091444,NaN,650.0,10.0
8608,22,78846,464,14,650-660,59,1,0,1,1.003839,NaN,650.0,10.0
8610,22,59317,464,14,650-660,59,1,0,1,0.833296,NaN,650.0,10.0


In [187]:
len(df_c_train) + len(df_c_predict) + len(final_df)
len(df)

9601

In [191]:
final_22 = pd.concat([df_c_train, df_c_predict]).sort_index()

final_22.to_csv("./ds_tractats/final_22.csv", index = False)

In [192]:
final_22

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
8526,22,48626,302,13,600,32,1,0,1,3.7503,NaN,600.0,0.0
8527,22,47921,302,13,600,32,1,0,1,7.3735,NaN,600.0,0.0
8528,22,5696,919,14,650-660,59,0,0,1,7.6200,NaN,650.0,10.0
8529,22,98814,919,14,650-660,32,1,0,1,3.3267,NaN,650.0,10.0
8530,22,98814,919,14,650-660,40,1,0,1,2.7724,NaN,650.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,1,0,1,3.6800,NaN,700.0,0.0
9597,22,58769,239,6,700,32,1,0,1,4.2500,NaN,700.0,0.0
9598,22,58769,239,6,700,59,1,0,1,4.0700,NaN,700.0,0.0
9599,22,88928,239,6,700,40,1,0,1,4.5727,NaN,700.0,0.0
